In [ ]:
import os
from pathlib import Path
import json
import csv
import numpy as np
from PIL import Image, ImageFile

In [ ]:
def get_features(label: ImageFile) -> dict:
    label_data = np.array(label)
    values, counts = np.unique(label_data, return_counts=True)
    return dict(zip(values, counts))

In [ ]:
def is_area_flooded(label: ImageFile, classes) -> int:
    width, height = label.size
    total_pixels = width * height
    features = get_features(label)

    flood_level = 0
    for feature in features:
        if feature == classes['Building-flooded'] or feature == classes['Road-flooded'] or feature == classes['Water']:
            flood_level = flood_level + features[feature]

    area_flooded = flood_level / total_pixels
    return 1 if area_flooded >= 0.3 else 0

In [ ]:
class StatsCollector():
    def __init__(self, stats_structure: dict):
        self.stats = stats_structure
    
    def get_stat(self, stat_path: str, separator: str = "."):
        keys = stat_path.split(separator)
        stat = self.stats

        try:
            for key in keys:
                # Check if the key represents a list index
                if key.isdigit():
                    key = int(key)  # Convert key to integer for list indexing
                stat = stat[key]  # Navigate through the structure (dict or list)
            return stat
        except (KeyError, IndexError, TypeError):
            return None  # Return None if any key/index is invalid

    def inc_stat(self, stat_path: str, separator: str = "."):
        keys = stat_path.split(separator)
        stat = self.stats

        try:
            for key in keys[:-1]:
                # Check if the key represents a list index
                if key.isdigit():
                    key = int(key)  # Convert key to integer for list indexing
                stat = stat[key]  # Navigate through the structure (dict or list)
            
            last_key = keys[-1]
            if last_key.isdigit():
                last_key = int(last_key)
            stat[last_key] = stat[last_key] + 1
            return True
        except (KeyError, IndexError, TypeError):
            return False  # Return None if any key/index is invalid

    def save_stats_to_csv(filename):
        pass

In [92]:
# create stats tree structure
with open("classes.json", "r") as j:
    classes = json.load(j)
    class_count_dict = {key: 0 for key in classes.keys()}

flood_stats = StatsCollector({
    "train": {
        "images": 0,
        "flooded": 0,
        "non_flooded": 0,
        "classes": class_count_dict
    },
    "test": {
        "images": 0,
        "flooded": 0,
        "non_flooded": 0,
        "classes": class_count_dict
    },
    "val": {
        "images": 0,
        "flooded": 0,
        "non_flooded": 0,
        "classes": class_count_dict
    }
})

In [97]:
print(flood_stats.get_stat("test.images"))
print(flood_stats.inc_stat("test.images"))
print(flood_stats.get_stat("test.images"))

4
True
5


In [ ]:
# set path variables
root_dir = Path("FloodNet_dataset/")
classes_json_path = Path("classes.json")
data_types = ["train", "test", "val"]

with open(classes_json_path, "r") as j:
    classes = json.load(j)

for data_type in data_types:
    fieldnames = ['image', 'is_flooded']

    with open(f'flood_{data_type}_rel_paths.csv', 'w', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        img_dir_path = Path(f"{root_dir}/{data_type}/image/")
        label_dir_path = Path(f"{root_dir}/{data_type}/label/")

        for f in os.listdir(label_dir_path):
            print(f"--------------- {f} ---------------")

            label_path = os.path.join(label_dir_path, f)
            label = Image.open(label_path)

            is_flooded = is_area_flooded(label, classes)
            
            image_name = f.replace("_lab.png", ".jpg")
            image_full_path = os.path.join(img_dir_path, image_name)
            image_rel_path = os.path.relpath(image_full_path, root_dir)
            
            print(f"image: {image_name}, is flooded: {is_flooded}")
            writer.writerow({fieldnames[0]: image_name, fieldnames[1]: is_flooded})
